In [1]:
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
import tkinter as tk
from pysrc.simplertplot.plotwidget import RTPlotWidget
from hello.hello3 import HelloApp, TrueError
from time import time
import numpy as np

In [3]:
ip = '192.168.1.85'
#ip = '71.189.82.196:86'

In [4]:
interval = 100
minutes = 2
max_pts=minutes*60/(interval/1000)
max_pts=int(max_pts)
max_pts

1200

In [5]:
class MyPlotWidget(RTPlotWidget):
    def _ma(self, a, n):
        rv = np.zeros(len(a))
        n //= 2
        if n < 1:
            n = 1
        for i in range(len(a)):
            rv[i] = np.mean(a[max(i-n, 0):i+n])
        return rv

    def _update_data(self):

        _plot_x_data = self._ma(self.x_data.get(), 3)
        _plot_y_data = self._ma(self.y_data.get(), 3)
        self.line.set_data(self.x_data.get(), self.y_data.get())
        self.subplot.relim()
        self.subplot.autoscale_view(True, True, True)
        lower, upper = self.subplot.get_ybound()
        self.subplot.set_ylim(lower, upper + (upper - lower) * 0.02, True, None)
        self._update()


In [6]:
root = tk.Tk()
root.withdraw()

p = RTPlotWidget(root, max_pts=max_pts)
p.tkcanvas.config(height=400, width=600)
max_pts

1200

In [7]:
h = HelloApp(ip)
h.login()
h.getMainValues()
pass

In [8]:
def timer():
    start = time()
    while True:
        current = time()
        yield current - start
get_time = timer().__next__

In [9]:
g_stop_tasks = False
g_after = None
def tk_task(tk, interval, f):
    global g_after, g_stop_tasks
    def fn(*args, **kw):
        try:
            return f(*args, **kw)
        finally:
            if not g_stop_tasks:
                g_after = tk.after(interval, fn)
            else:
                tk.after_cancel(g_after)
    g_after = tk.after(interval, fn)

In [10]:
def update():
    mv = h.getMainValues()
    ag = mv['agitation']
    pv = ag['pv']
    t = get_time()
    p.add_xy(t, pv)
    update_labels(mv)

In [11]:
def printdir(o):
    for a in dir(o):
        if not a.startswith("_"):
            print(a, getattr(o, a))

In [12]:
def conf2(ev):
    if ev.widget == root:
        p.tkcanvas.config(height=ev.height, width=ev.width-rbtn.winfo_width())

In [13]:
class ILabel(tk.Label):
    def __init__(self, master, name, value=0, fmt="%.2f"):
        super().__init__(master)
        self.fmt = fmt
        self.name = name
        self.set_value(value)
        
    def set_value(self, val):
        val = self.fmt % val
        self.config(text="%s: %s" % (self.name, val))
   
labels = {}
def mk_label(name, val, row):
    l = ILabel(root, name, val)
    labels[name] = l
    l.grid(row=row, column=1)
    
mk_label("PV", 0, 1)
mk_label("OP", 0, 2)
    
def update_labels(mv):
    labels['PV'].set_value(mv['agitation']['pv'])
    labels['OP'].set_value(mv['agitation']['output'])

    

In [ ]:
p.grid(rowspan=20, column=0, row=0)    
def reset():
    p.reset()
    global get_time
    get_time = timer().__next__
rbtn = tk.Button(root, text="Reset", command=reset)
rbtn.grid(column=1, row=0, sticky="ew")
root.bind("<Configure>", conf2)
p.set_xaxis_text("Time (s)")
p.set_yaxis_text("DO PV (%)")
def on_closing():
    from tkinter import messagebox
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        global g_stop_tasks
        g_stop_tasks = True
        root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.wm_deiconify()
tk_task(root, interval, update)
root.mainloop()

In [ ]:
import numpy as np
a=np.zeros(20)

In [ ]:
3//2

In [ ]:
a=np.array